In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
import tensorflow as tf

In [2]:
df = pd.read_csv("Epileptic Seizure Recognition.csv")
df.head()

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [3]:
df['y'] = df['y'].replace({2: 0, 3: 0, 4: 0, 5: 0})
X = df.loc[:, df.columns.difference(['y', 'Unnamed'])]
y = df['y']


In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)



In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)


In [6]:
# SVM with cost-sensitive learning
svm = SVC(probability=True, class_weight={0: 1, 1: 3}, random_state=42)
svm.fit(X_train, y_train)
svm_probs = svm.predict_proba(X_test)[:, 1]
svm_preds = svm.predict(X_test)


In [7]:
# XGBoost
#xgb = XGBClassifier(eval_metric='logloss')
#xgb.fit(X_train, y_train)
#xgb_probs = xgb.predict_proba(X_test)[:, 1]
#xgb_preds = xgb.predict(X_test)

# Count class imbalance
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# XGBoost with cost-sensitive learning
xgb = XGBClassifier(
                    eval_metric='aucpr',
                    scale_pos_weight=scale_pos_weight,
                    random_state=42)

xgb.fit(X_train, y_train)
xgb_probs = xgb.predict_proba(X_test)[:, 1]
xgb_preds = xgb.predict(X_test)

In [8]:
from lightgbm import LGBMClassifier
import numpy as np

# Make sure y_train is also a NumPy array
y_train_np = np.array(y_train)

# Calculate scale_pos_weight
neg, pos = np.bincount(y_train_np)
scale_pos_weight = neg / pos

# Initialize LightGBM
lgbm = LGBMClassifier(
    objective='binary',
    scale_pos_weight=scale_pos_weight,
    metric='auc',
    random_state=42,
    n_estimators=100
)

# Fit model (X_train and y_train are already NumPy arrays)
lgbm.fit(X_train, y_train_np)

# Predict probabilities
lgbm_probs = lgbm.predict_proba(X_test)[:, 1]


[LightGBM] [Info] Number of positive: 1840, number of negative: 7360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45390
[LightGBM] [Info] Number of data points in the train set: 9200, number of used features: 178
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200000 -> initscore=-1.386294
[LightGBM] [Info] Start training from score -1.386294


D:\Anaconda\envs\mldl_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(class_weight={0: 1, 1: 3}, max_iter=1000)
log_reg.fit(X_train, y_train)
log_reg_probs = log_reg.predict_proba(X_test)[:, 1]
log_reg_preds = log_reg.predict(X_test)

In [10]:
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_probs = knn.predict_proba(X_test)[:, 1]
knn_preds = knn.predict(X_test)

In [11]:
# ANN
from tensorflow.keras import Input
ann = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
ann_probs = ann.predict(X_test).flatten()
ann_preds = (ann.predict(X_test) > 0.5).astype("int32")

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [12]:
# Reshape input for RNN, LSTM, CNN+BiLSTM
X_train_seq = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_seq = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [13]:
# GRU
from tensorflow.keras.layers import GRU
gru = Sequential([
    Input(shape=(X_train_seq.shape[1], 1)),
    GRU(64),
    Dense(1, activation='sigmoid')
])
gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gru.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)
gru_probs = gru.predict(X_test_seq).flatten()
gru_preds = (gru_probs > 0.5).astype("int32")

72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step


In [14]:
# LSTM
lstm = Sequential([
    Input(shape=(X_train_seq.shape[1], 1)),
    LSTM(64),
    Dense(1, activation='sigmoid')
])
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)
lstm_probs = lstm.predict(X_test_seq).flatten()
lstm_preds = (lstm_probs > 0.5).astype("int32")

72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step


In [15]:
# RNN
rnn = Sequential([
    Input(shape=(X_train_seq.shape[1], 1)),
    SimpleRNN(64),
    Dense(1, activation='sigmoid')
])
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)
rnn_probs = rnn.predict(X_test_seq).flatten()
rnn_preds = (rnn_probs > 0.5).astype("int32")

72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [16]:
# CNN + BiLSTM
input_layer = Input(shape=(X_train_seq.shape[1], 1))
x = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(64))(x)
output = Dense(1, activation='sigmoid')(x)
cnn_bilstm_model = Model(inputs=input_layer, outputs=output)
cnn_bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_bilstm_model.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)
cnn_bilstm_probs = cnn_bilstm_model.predict(X_test_seq).flatten()
cnn_biltsm_preds = (cnn_bilstm_probs > 0.5).astype("int32")

72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step


In [17]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout, Input, Flatten
from tensorflow.keras.models import Model

# Transformer Encoder Block
def transformer_encoder_block(inputs, head_size=64, num_heads=2, ff_dim=128, dropout=0.1):
    # Multi-head self-attention
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + inputs)  # Residual

    # Feed-forward network
    ff = Dense(ff_dim, activation="relu")(x)
    ff = Dense(inputs.shape[-1])(ff)
    ff = Dropout(dropout)(ff)
    x = LayerNormalization(epsilon=1e-6)(x + ff)  # Residual
    return x


In [18]:
# Input
transformer_input = Input(shape=(X_train_seq.shape[1], 1))

# Transformer block
x = transformer_encoder_block(transformer_input)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
transformer_output = Dense(1, activation='sigmoid')(x)

# Model
transformer_model = Model(inputs=transformer_input, outputs=transformer_output)
transformer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
transformer_model.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)

# Predictions
transformer_probs = transformer_model.predict(X_test_seq).flatten()
transformer_preds = (transformer_probs > 0.5).astype("int32")


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


In [19]:
from tensorflow.keras.layers import Add

def resnet_block(inputs, filters, kernel_size, stride=1):
    x = Conv1D(filters, kernel_size, strides=stride, padding='same', activation='relu')(inputs)
    x = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    shortcut = Conv1D(filters, 1, strides=stride, padding='same')(inputs)  # match dimensions
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x


In [20]:
from tensorflow.keras.layers import GlobalAveragePooling1D, Activation

resnet_input = Input(shape=(X_train_seq.shape[1], 1))

x = resnet_block(resnet_input, filters=64, kernel_size=3)
x = resnet_block(x, filters=64, kernel_size=3)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
resnet_output = Dense(1, activation='sigmoid')(x)

resnet_model = Model(inputs=resnet_input, outputs=resnet_output)
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
resnet_model.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)

resnet_probs = resnet_model.predict(X_test_seq).flatten()
resnet_preds = (resnet_probs > 0.5).astype("int32")


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


In [21]:
attn_input = Input(shape=(X_train_seq.shape[1], 1))
x = LSTM(64, return_sequences=True)(attn_input)
x = MultiHeadAttention(num_heads=2, key_dim=64)(x, x)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
attn_output = Dense(1, activation='sigmoid')(x)
attn_model = Model(inputs=attn_input, outputs=attn_output)
attn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
attn_model.fit(X_train_seq, y_train, epochs=20, batch_size=32, verbose=0)
attn_probs = attn_model.predict(X_test_seq).flatten()

72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step


In [22]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Convert data to float32 (TabNet requires this)
X_train_tabnet = X_train.astype(np.float32)
X_test_tabnet = X_test.astype(np.float32)
y_train_tabnet = y_train.values if hasattr(y_train, 'values') else y_train
y_test_tabnet = y_test.values if hasattr(y_test, 'values') else y_test

# TabNet model
tabnet = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=0
)

# Fit model
tabnet.fit(
    X_train=X_train_tabnet,
    y_train=y_train_tabnet,
    eval_set=[(X_test_tabnet, y_test_tabnet)],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0
)

# Predict probabilities and hard labels
tabnet_probs = tabnet.predict_proba(X_test_tabnet)[:, 1]
tabnet_preds = (tabnet_probs > 0.5).astype("int32")



Early stopping occurred at epoch 25 with best_epoch = 15 and best_val_0_accuracy = 0.93


D:\Anaconda\envs\mldl_env\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [31]:
# Ensemble Averaging (Soft Voting)
avg_probs = (svm_probs + xgb_probs + lgbm_probs + ann_probs + lstm_probs + gru_probs + rnn_probs + cnn_bilstm_probs + resnet_probs + attn_probs + transformer_probs ) / 11
ensemble_preds = (avg_probs > 0.4).astype("int32")

In [32]:
# Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("\n--- Ensemble Model Averaging Results ---")
print(classification_report(y_test, ensemble_preds))
print("Confusion Matrix:\n", confusion_matrix(y_test, ensemble_preds))
print("Accuracy:", accuracy_score(y_test, ensemble_preds))



--- Ensemble Model Averaging Results ---
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1840
           1       0.95      0.96      0.96       460

    accuracy                           0.98      2300
   macro avg       0.97      0.98      0.97      2300
weighted avg       0.98      0.98      0.98      2300

Confusion Matrix:
 [[1819   21]
 [  17  443]]
Accuracy: 0.9834782608695652


In [33]:
from sklearn.metrics import confusion_matrix

# Assuming y_test and ensemble_preds are already defined
cm = confusion_matrix(y_test, ensemble_preds)

# Extract True Positives and False Negatives
TP = cm[1][1]
FN = cm[1][0]

# Calculate Sensitivity
sensitivity = TP / (TP + FN)
print(f"Sensitivity (Recall): {sensitivity:.4f}")


Sensitivity (Recall): 0.9630
